## Get the emails

In [ ]:
import imaplib
import email
import yaml
import concurrent.futures
import pandas as pd

### Open the file that have the gmail credentials

In [ ]:
with open('credentials.yaml') as f:
    credentials = f.read()

### Get the User and Password

In [ ]:
my_credentials = yaml.load(credentials, Loader = yaml.FullLoader)
user, password = my_credentials['user'], my_credentials['password']

### Connect to Gmail

In [ ]:
imap_url = 'imap.gmail.com'
my_email = imaplib.IMAP4_SSL(imap_url)
my_email.login(user, password)

('OK', [b'demik.freitast2d18@gmail.com authenticated (Success)'])

### Number of emails in my inbox

In [ ]:
my_email.select('inbox')[1][0].decode()

'18567'

### Import emails from inbox and then put them in a list

In [ ]:
emails = my_email.search(None, 'ALL')
emails_ids = emails[1][0].decode().split()

In [ ]:
def fetch(id):
  with concurrent.futures.ThreadPoolExecutor() as executor:
    data = executor.submit(my_email.fetch, str(id), '(RFC822)') # Get the message informations (Message, emails, ids, if errors, etc...)
    return data.result()[1][0][1] # Return only message and e-mail

## Test: fetch(1)

In [ ]:
# Get the message and the email informations

emails_list = []

for i in emails_ids:
      msg = email.message_from_string(str(fetch(i),'ISO-8859-1')) # Transform the e-mails 
      emails_list.append({'Date': msg['Date'], 'From': msg['From'], 'Subject': msg['Subject']})

KeyboardInterrupt: ignored

### Put email data in a dataframe

In [ ]:
df = pd.DataFrame(emails_list, columns=['Date', 'From', 'Subject'])
df

In [ ]:
df1 = df.copy()
df1

In [ ]:
df1.dtypes

Date       object
From       object
Subject    object
dtype: object

### Resolve encoding issues

In [ ]:
pd.set_option('max_colwidth', None)

#### Convert column encode 'From'

In [ ]:
df1['From'] = df1['From'].apply(lambda x: email.header.decode_header(x)[0][0])

In [ ]:
df1['From'] = df1['From'].apply(lambda x: x.decode('utf-8', 'replace') if isinstance(x, bytes) else x)

#### Convert column encode 'Subject'

In [ ]:
df1['Subject'] = df1['Subject'].apply(lambda x: email.header.decode_header(x)[0][0])

In [ ]:
df1['Subject'] = df1['Subject'].apply(lambda x: x.decode('utf-8', 'replace') if isinstance(x, bytes) else x)

In [ ]:
df1.head(10)

### Clear the 'From' column

#### Get the sender's email address

In [ ]:
df1['Email'] = df1['From'].str.extract(r'<(.+)>')

In [ ]:
df1.head(10)

#### Get the sender's name

In [ ]:
df1['Sender'] = df1['From'].str.extract(r'(?:"|^)(.*?)(?:"|\s)(?:\s*<|$)')

#### Delete From column (useless)

In [ ]:
del df1['From']

In [ ]:
df1.sample(20)

### Reorder columns

In [ ]:
df1 = df1[['Date', 'Sender', 'Email', 'Subject']]
df1

### Convert data format

#### Extract the usefull date

In [ ]:
df1['Date'] = df1['Date'].str.extract(r'(\d+ \w{3} \d+)')

In [ ]:
df1['Date'].isnull().sum()

0

#### Verify all the month abbreviations

In [ ]:
df1['Date'].str.extract(r'(\w{3})').squeeze().unique()

array(['Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May',
       'Jun', 'Jul', 'Aug', 'ago'], dtype=object)

#### For some reason there is a month like 'ago' instead of 'Aug', it should be replaced.

In [ ]:
df1['Date'] = df1['Date'].str.replace('ago', 'Aug')

In [ ]:
df1

In [ ]:
df1['Date'] = pd.to_datetime(df1['Date'], format='%d %b %Y').dt.strftime('%d/%m/%Y')
df1

### Drop the columns 

In [ ]:
df1 = df1.dropna(subset=['Sender', 'Email'], thresh=2)

### Save the dataframe to CSV

In [ ]:
df1.to_csv('Emails_Dataset.csv', sep=';', encoding='utf-8')